In [39]:
import torch
import numpy as np

project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

from models.Paper import CNN
from InformationPlane import TensorKernel, MutualInformation

In [49]:
a = torch.rand((4,1,3,3), dtype=torch.float32)
b = torch.zeros((4,3), dtype=torch.float32)
b[0,1] = 1; b[1,2] = 1; b[2,0] = 1; b[3,1] = 1

print(a)
print(b)

tensor([[[[0.7525, 0.9846, 0.4575],
          [0.5977, 0.4467, 0.9081],
          [0.7331, 0.7528, 0.0785]]],


        [[[0.0756, 0.6663, 0.2696],
          [0.1421, 0.9024, 0.7948],
          [0.9124, 0.9937, 0.5457]]],


        [[[0.7959, 0.6191, 0.2340],
          [0.9489, 0.7562, 0.9825],
          [0.5304, 0.0863, 0.2953]]],


        [[[0.7853, 0.5786, 0.6021],
          [0.3281, 0.0440, 0.3754],
          [0.0736, 0.7651, 0.5047]]]])
tensor([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.]])


In [50]:
a_flatten = torch.flatten(a, 1)

# Tensor kernels definitions

def RBF(x, y, sigma):
    distance = torch.dist(x,y)
    return torch.exp(-(distance**2)/(sigma**2))

def RBF_2(x, sigma):
    distance = torch.cdist(x, x)
    return torch.exp(-(distance**2)/(sigma**2))

# Ambas funciones dan el mismo resultado, lo doy por bueno


In [77]:
rbf_a = RBF_2(a_flatten, 8)/len(a_flatten)
rbf_b = RBF_2(b, 0.9)/len(b)

print(rbf_a)
print(rbf_b)

tensor([[0.2500, 0.2449, 0.2463, 0.2449],
        [0.2449, 0.2500, 0.2413, 0.2411],
        [0.2463, 0.2413, 0.2500, 0.2419],
        [0.2449, 0.2411, 0.2419, 0.2500]])
tensor([[0.2500, 0.0212, 0.0212, 0.2500],
        [0.0212, 0.2500, 0.0212, 0.0212],
        [0.0212, 0.0212, 0.2500, 0.0212],
        [0.2500, 0.0212, 0.0212, 0.2500]])


In [66]:
k_0 = torch.zeros((len(a_flatten), len(a_flatten)))
for i in range(len(a_flatten)):
    for j in range(len(a_flatten)):
        k_0[i,j] = RBF(a_flatten[i], a_flatten[j], 8)
        
print(k_0)

tensor([[1.0000, 0.9794, 0.9854, 0.9796],
        [0.9794, 1.0000, 0.9653, 0.9643],
        [0.9854, 0.9653, 1.0000, 0.9675],
        [0.9796, 0.9643, 0.9675, 1.0000]])


In [53]:
k_1 = torch.zeros((len(b), len(b)))
for i in range(len(b)):
    for j in range(len(b)):
        k_1[i,j] = RBF(b[i], b[j], 0.1)
        
print(k_1)

tensor([[1., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.]])


# Kernel Aligment Loss

In [68]:
def kernelAligmentLoss(x, y):
    return ((torch.sum(x*y))/(torch.norm(x) * torch.norm(y))).item()

In [69]:
kernelAligmentLoss(rbf_a, rbf_b)

0.7019651532173157

# Optimize sigma value

In [70]:
mean_distance = torch.tensor([torch.dist(a_flatten[i-1], a_flatten[i]) for i in range(1, len(a_flatten))]).mean()

distance = torch.cdist(a_flatten, a_flatten)
mean_distance_v2 = distance[distance != 0].mean()

print(a_flatten)
print(mean_distance)
print(mean_distance_v2)

possible_sigma_values = np.arange(mean_distance*0.1, mean_distance*10, (mean_distance*10 - mean_distance*0.1)/75)
possible_sigma_values_v2 = np.arange(mean_distance_v2*0.1, mean_distance_v2*10, (mean_distance_v2*10 - mean_distance_v2*0.1)/75)

print(possible_sigma_values)
print(possible_sigma_values_v2)

# print(possible_sigma_values_v2.tolist())

test = list(map(lambda sigma: RBF_2(a_flatten, sigma), possible_sigma_values_v2.tolist()))
print(test[70])

label_kernel = rbf_b
result = np.array(list(map(lambda x: kernelAligmentLoss(x, label_kernel), test)))
print(result)

where = np.argwhere(result == result.max()).item()
print(where)

best_sigma_value = possible_sigma_values_v2[where]
print(best_sigma_value)

tensor([[0.7525, 0.9846, 0.4575, 0.5977, 0.4467, 0.9081, 0.7331, 0.7528, 0.0785],
        [0.0756, 0.6663, 0.2696, 0.1421, 0.9024, 0.7948, 0.9124, 0.9937, 0.5457],
        [0.7959, 0.6191, 0.2340, 0.9489, 0.7562, 0.9825, 0.5304, 0.0863, 0.2953],
        [0.7853, 0.5786, 0.6021, 0.3281, 0.0440, 0.3754, 0.0736, 0.7651, 0.5047]])
tensor(1.3704)
tensor(1.2929)
[ 0.13703752  0.31792703  0.49881655  0.67970607  0.86059558  1.0414851
  1.22237462  1.40326414  1.58415365  1.76504317  1.94593269  2.1268222
  2.30771172  2.48860124  2.66949075  2.85038027  3.03126979  3.21215931
  3.39304882  3.57393834  3.75482786  3.93571737  4.11660689  4.29749641
  4.47838593  4.65927544  4.84016496  5.02105448  5.20194399  5.38283351
  5.56372303  5.74461254  5.92550206  6.10639158  6.2872811   6.46817061
  6.64906013  6.82994965  7.01083916  7.19172868  7.3726182   7.55350772
  7.73439723  7.91528675  8.09617627  8.27706578  8.4579553   8.63884482
  8.81973433  9.00062385  9.18151337  9.36240289  9.5432924

In [57]:
rbf_c = TensorKernel.RBF(a_flatten, 8)
I = MutualInformation(rbf_c, rbf_b)
print(rbf_c)
rbf_c[0, 0] = 0
print(rbf_c)
print(I.input_kernel)

tensor([[1.0000, 0.9794, 0.9854, 0.9796],
        [0.9794, 1.0000, 0.9653, 0.9643],
        [0.9854, 0.9653, 1.0000, 0.9675],
        [0.9796, 0.9643, 0.9675, 1.0000]])
tensor([[0.0000, 0.9794, 0.9854, 0.9796],
        [0.9794, 1.0000, 0.9653, 0.9643],
        [0.9854, 0.9653, 1.0000, 0.9675],
        [0.9796, 0.9643, 0.9675, 1.0000]])
tensor([[0.0000, 0.9794, 0.9854, 0.9796],
        [0.9794, 1.0000, 0.9653, 0.9643],
        [0.9854, 0.9653, 1.0000, 0.9675],
        [0.9796, 0.9643, 0.9675, 1.0000]])


In [58]:
print(I.optimizeSigmaValue(a_flatten))

1.153242066502571


In [59]:
y = None
x = 0
x = 1 if not(y is None) else x
print(x)

0


In [60]:
print(rbf_a)
print(torch.eig(rbf_a))

tensor([[1.0000, 0.9794, 0.9854, 0.9796],
        [0.9794, 1.0000, 0.9653, 0.9643],
        [0.9854, 0.9653, 1.0000, 0.9675],
        [0.9796, 0.9643, 0.9675, 1.0000]])
torch.return_types.eig(
eigenvalues=tensor([[3.9208, 0.0000],
        [0.0100, 0.0000],
        [0.0329, 0.0000],
        [0.0363, 0.0000]]),
eigenvectors=tensor([]))


In [73]:
import numpy as np
from numpy import linalg as LA
# A = np.array([[1,2,3],[3,2,1],[1,0,-1]])
A = rbf_a.numpy()
print(A)

w, v = LA.eig(A)
print(w)

print(v)


[[0.25       0.24485293 0.24634588 0.24488765]
 [0.24485293 0.25       0.24131835 0.24107365]
 [0.24634588 0.24131835 0.25       0.24188475]
 [0.24488765 0.24107365 0.24188475 0.25      ]]
[0.9801942  0.00250741 0.00823296 0.0090654 ]
[[-0.5030177  -0.8530471  -0.13365872  0.03767203]
 [-0.49849018  0.23751526  0.12795079 -0.8238463 ]
 [-0.49967963  0.41680747 -0.69081694  0.31522015]
 [-0.49879956  0.2053502   0.6989534   0.46956834]]


In [126]:
def matrixRenyiEntropy(A : np.array):
    w, _ = LA.eig(A)
    epsilon = 1e-10
    w += epsilon
    entropy = -np.sum(w * np.log2(w))
    
    return entropy

def jointRenyiEntropy(x, y):
    aux = x*y
    return matrixRenyiEntropy(aux/np.trace(aux))
    

In [129]:
o = matrixRenyiEntropy(rbf_a.numpy())
p = matrixRenyiEntropy(rbf_b.numpy())
q = jointRenyiEntropy(rbf_a.numpy(), rbf_b.numpy())
print(o)
print(p)
print(q)

print(o + p - q)

0.16847238
1.490485
1.5321074
0.12685001


In [124]:
print(rbf_a)
print(torch.trace(rbf_a))
print(rbf_b)
aux = rbf_a * rbf_b
print(aux)
print(torch.trace(aux))
print(aux/torch.trace(aux))

tensor([[0.2500, 0.2449, 0.2463, 0.2449],
        [0.2449, 0.2500, 0.2413, 0.2411],
        [0.2463, 0.2413, 0.2500, 0.2419],
        [0.2449, 0.2411, 0.2419, 0.2500]])
tensor(1.)
tensor([[0.2500, 0.0212, 0.0212, 0.2500],
        [0.0212, 0.2500, 0.0212, 0.0212],
        [0.0212, 0.0212, 0.2500, 0.0212],
        [0.2500, 0.0212, 0.0212, 0.2500]])
tensor([[0.0625, 0.0052, 0.0052, 0.0612],
        [0.0052, 0.0625, 0.0051, 0.0051],
        [0.0052, 0.0051, 0.0625, 0.0051],
        [0.0612, 0.0051, 0.0051, 0.0625]])
tensor(0.2500)
tensor([[0.2500, 0.0207, 0.0209, 0.2449],
        [0.0207, 0.2500, 0.0204, 0.0204],
        [0.0209, 0.0204, 0.2500, 0.0205],
        [0.2449, 0.0204, 0.0205, 0.2500]])
